In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 라이브러리 로드

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import seaborn as sns
import os

In [4]:
# 폰트 설정
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()
    # colab 사용자는 system_name이 'Linux'로 확인

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont
        
        font_family = "NanumBarunGothic"
    return font_family

In [5]:
# 폰트 적용
font_family = get_font_family()
plt.rc("font", family=font_family)

# 마이너스 폰트 설정
plt.rc("axes", unicode_minus=False)

# 그래프에 retina display 적용
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


### 파일 불러오기

In [6]:
path_dir = '/content/gdrive/MyDrive/Data Analysis/Aiffel/HAC/03/data'

In [7]:
univ_data = pd.read_excel(path_dir + '/전국 대학교 현황_201910_주소포함.xlsx', header=5)
univ_data.shape

(1876, 13)

In [8]:
univ_data.head()

,연도,학교종류,시도,학교명,학교명(영문),본분교,학교상태,설립,우편번호,주소,전화번호,팩스번호,홈페이지
0,2019,대학교,강원,강릉원주대학교,Gangneung-Wonju National University,본교,기존,국립,25457,"강원도 강릉시 죽헌길 7 (지변동, 강릉원주대학교)",033-640-7001,033-643-7110,www.gwnu.ac.kr
1,2019,대학교,강원,강릉원주대학교,Gangneung-Wonju National University,제2캠퍼스,기존,국립,26403,"강원도 원주시 흥업면 남원로 150 (흥업리, 강릉원주대학교원주캠퍼스)",033-760-8114,033-760-8019,www.gwnu.ac.kr
2,2019,일반대학원,강원,강릉원주대학교대학원,Gangneung-Wonju National University Graduate S...,본교,기존,국립,25457,"강원도 강릉시 죽헌길 7 (지변동, 강릉원주대학교)",033-640-2077,NaN,www.gwnu.ac.kr
3,2019,일반대학원,강원,강릉원주대학교대학원,Gangneung-Wonju National University Graduate S...,제2캠퍼스,기존,국립,26403,"강원도 원주시 흥업면 남원로 150 (흥업리, 강릉원주대학교원주캠퍼스)",033-640-2077,NaN,www.gwnu.ac.kr
4,2019,특수대학원,강원,강릉원주대학교경영정책과학대학원,Gangneung Wonju Graduate School of Management ...,본교,기존,국립,25457,"강원도 강릉시 죽헌길 7 (지변동, 강릉원주대학교)",033-640-2081,NaN,www.gwnu.ac.kr


In [9]:
univ_data.columns

Index(['연도', '학교종류', '시도', '학교명', '학교명(영문)', '본분교', '학교상태', '설립', '우편번호', '주소',
       '전화번호', '팩스번호', '홈페이지'],
      dtype='object')

In [10]:
# 필요한 컬럼만 보기
univ_data = univ_data[['학교종류', '시도', '학교명', '본분교', '학교상태', '설립', '우편번호', '주소']]
univ_data.head()

,학교종류,시도,학교명,본분교,학교상태,설립,우편번호,주소
0,대학교,강원,강릉원주대학교,본교,기존,국립,25457,"강원도 강릉시 죽헌길 7 (지변동, 강릉원주대학교)"
1,대학교,강원,강릉원주대학교,제2캠퍼스,기존,국립,26403,"강원도 원주시 흥업면 남원로 150 (흥업리, 강릉원주대학교원주캠퍼스)"
2,일반대학원,강원,강릉원주대학교대학원,본교,기존,국립,25457,"강원도 강릉시 죽헌길 7 (지변동, 강릉원주대학교)"
3,일반대학원,강원,강릉원주대학교대학원,제2캠퍼스,기존,국립,26403,"강원도 원주시 흥업면 남원로 150 (흥업리, 강릉원주대학교원주캠퍼스)"
4,특수대학원,강원,강릉원주대학교경영정책과학대학원,본교,기존,국립,25457,"강원도 강릉시 죽헌길 7 (지변동, 강릉원주대학교)"


In [11]:
univ_data['학교종류'].unique()

array(['대학교', '일반대학원', '특수대학원', '전문대학원', '교육대학', '산업대학', '방송통신대학', '기술대학',
       '사내대학(전문)', '사내대학(대학)', '전공대학', '전문대학(2년제)', '전문대학(3년제)',
       '전문대학(4년제)', '각종대학(대학)', '원격대학(전문)', '사이버대학(전문)', '사이버대학(대학)',
       '원격대학(대학)', '기능대학'], dtype=object)

대학원, 사이버대학, 사내대학 등은 우리가 필요로 하는 정보는 아니다.

In [12]:
# 애매한 학교 종류 찾아보기
univ_data[univ_data['학교종류'] == '전공대학']

,학교종류,시도,학교명,본분교,학교상태,설립,우편번호,주소
1609,전공대학,서울,국제예술대학교,본교,기존,사립,6051,"서울특별시 강남구 도산대로30길 47 (논현동, 국제예술대학교)"
1610,전공대학,서울,백석예술대학교,본교,기존,사립,6695,"서울특별시 서초구 방배로9길 23 (방배동, 백석예술대학교)"
1611,전공대학,서울,정화예술대학교,본교,기존,사립,4631,"서울특별시 중구 퇴계로16길 21 (남산동1가, 정화예술대학교명동캠퍼스본관)"
1612,전공대학,서울,정화예술대학교,제2캠퍼스,기존,사립,4633,"서울특별시 중구 퇴계로8길 73 (회현동1가, 정화예술대학교남산캠퍼스)"


In [13]:
# 필요한 컬럼만 사용하기
univ_type = ['대학교', '교육대학', '산업대학', '전공대학', '전문대학(2년제)', '전문대학(3년제)',
             '전문대학(4년제)', '각종대학(대학)','기능대학']

In [14]:
# '학교상태' 컬럼 unique 값
univ_data['학교상태'].unique()

array(['기존', '폐교', '학교명 변경', '신설'], dtype=object)

In [15]:
univ_data = univ_data.loc[univ_data['학교종류'].isin(univ_type)]  # 대학원, 사이버대학, 사내대학 등 제외
univ_data = univ_data[univ_data['학교상태'] != '폐교']  # 학교상태가 폐교가 아닌 학교만
univ_data.shape

(397, 8)

In [16]:
univ_data['시군구명'] = univ_data['주소'].str.split(" ", expand=True)[0] + " " + univ_data['주소'].str.split(" ", expand=True)[1]
univ_data['읍면동명'] = univ_data['주소'].str.split(" ", expand=True)[2]
univ_data.head()

,학교종류,시도,학교명,본분교,학교상태,설립,우편번호,주소,시군구명,읍면동명
0,대학교,강원,강릉원주대학교,본교,기존,국립,25457,"강원도 강릉시 죽헌길 7 (지변동, 강릉원주대학교)",강원도 강릉시,죽헌길
1,대학교,강원,강릉원주대학교,제2캠퍼스,기존,국립,26403,"강원도 원주시 흥업면 남원로 150 (흥업리, 강릉원주대학교원주캠퍼스)",강원도 원주시,흥업면
10,대학교,강원,강원대학교,본교,기존,국립,24341,"강원도 춘천시 강원대학길 1 (효자동, 강원대학교)",강원도 춘천시,강원대학길
11,대학교,강원,강원대학교,제2캠퍼스,기존,국립,25913,"강원도 삼척시 중앙로 346 (교동, 강원대학교삼척캠퍼스)",강원도 삼척시,중앙로
23,대학교,대구,경북대학교,본교,기존,국립,41566,"대구광역시 북구 대학로 80 (산격동, 경북대학교)",대구광역시 북구,대학로


In [17]:
univ_data.loc[univ_data['시군구명'].str.contains('세종'), '시군구명']

437     세종특별자치시 조치원읍
603      세종특별자치시 전의면
1405    세종특별자치시 조치원읍
1713     세종특별자치시 장군면
Name: 시군구명, dtype: object

In [18]:
univ_data['시군구명'] = univ_data['시군구명'].replace(['세종특별자치시 조치원읍','세종특별자치시 전의면','세종특별자치시 장군면'], '세종특별자치시 세종시')

In [19]:
univ_data.loc[univ_data['시군구명'].str.contains('세종'), '시군구명']

437     세종특별자치시 세종시
603     세종특별자치시 세종시
1405    세종특별자치시 세종시
1713    세종특별자치시 세종시
Name: 시군구명, dtype: object

In [20]:
socar_region = ['경기도 성남시', '경기도 고양시', '경기도 안양시', '전라북도 전주시']

In [21]:
univ_data.loc[univ_data['시군구명'].isin(socar_region)]  # socar_region에 속하는 데이터에만 읍면동 정보를 붙여줘야 한다

,학교종류,시도,학교명,본분교,학교상태,설립,우편번호,주소,시군구명,읍면동명
139,대학교,전북,전북대학교,본교,기존,국립,54896,"전라북도 전주시 덕진구 백제대로 567 (금암동, 전북대학교)",전라북도 전주시,덕진구
383,대학교,경기,가천대학교,본교,기존,사립,13120,"경기도 성남시 수정구 성남대로 1342 (복정동, 가천대학교)",경기도 성남시,수정구
823,대학교,경기,성결대학교,본교,기존,사립,14097,"경기도 안양시 만안구 성결대학로 53 (안양동, 성결대학교)",경기도 안양시,만안구
984,대학교,경기,안양대학교,본교,기존,사립,14028,"경기도 안양시 만안구 삼덕로37번길 22 (안양동, 안양대학교)",경기도 안양시,만안구
1082,대학교,경기,을지대학교,제2캠퍼스,기존,사립,13135,"경기도 성남시 수정구 산성대로 553 (양지동, 을지대학교)",경기도 성남시,수정구
1133,대학교,전북,전주대학교,본교,기존,사립,55069,"전라북도 전주시 완산구 천잠로 303 (효자동2가, 전주대학교)",전라북도 전주시,완산구
1270,대학교,경기,한국항공대학교,본교,기존,사립,10540,"경기도 고양시 덕양구 항공대학로 76 (화전동, 한국항공대학교)",경기도 고양시,덕양구
1457,대학교,전북,예수대학교,본교,기존,사립,54989,"전라북도 전주시 완산구 서원로 383 (중화산동1가, 예수대학교)",전라북도 전주시,완산구
1562,교육대학,경기,경인교육대학교,제2캠퍼스,기존,국립,13910,"경기도 안양시 만안구 삼막로 155 (석수동, 경인교육대학교)",경기도 안양시,만안구
1564,교육대학,전북,전주교육대학교,본교,기존,국립,55101,"전라북도 전주시 완산구 서학로 50 (동서학동, 전주교육대학교)",전라북도 전주시,완산구


In [22]:
# 읍변동 정보를 붙여주는 lambda 함수
univ_data['위치'] = univ_data.apply(lambda x : x['시군구명']+" "+x['읍면동명'] if x['시군구명'] in socar_region else x['시군구명'], axis = 1)

In [23]:
univ_data[univ_data['위치'] == '전라북도 전주시 덕진구']

,학교종류,시도,학교명,본분교,학교상태,설립,우편번호,주소,시군구명,읍면동명,위치
139,대학교,전북,전북대학교,본교,기존,국립,54896,"전라북도 전주시 덕진구 백제대로 567 (금암동, 전북대학교)",전라북도 전주시,덕진구,전라북도 전주시 덕진구
1845,전문대학(3년제),전북,한국농수산대학,본교,기존,특별법국립,54874,"전라북도 전주시 덕진구 콩쥐팥쥐로 1515 (중동, 한국농수산대학)",전라북도 전주시,덕진구,전라북도 전주시 덕진구


In [24]:
socar_region = ['경기도 하남시', '경기도 성남시 분당구', '경기도 안양시 만안구', '경기도 고양시 덕양구', '경기도 남양주시',
       '전라북도 전주시 완산구', '경기도 안양시 동안구', '경기도 의정부시', '경기도 광명시', '경기도 고양시 일산서구',
       '경기도 성남시 수정구', '경기도 화성시', '경기도 성남시 중원구', '울산광역시 남구', '경기도 김포시', '울산광역시 중구',
       '경기도 고양시 일산동구', '세종특별자치시 세종시', '울산광역시 북구', '부산광역시 강서구', '울산광역시 울주군', '경기도 구리시',
       '전라북도 전주시 덕진구', '울산광역시 동구', '경기도 양평군']

In [25]:
len(socar_region)

25

In [26]:
socar_region_univ = univ_data.loc[univ_data['위치'].isin(socar_region)]
socar_region_univ

,학교종류,시도,학교명,본분교,학교상태,설립,우편번호,주소,시군구명,읍면동명,위치
139,대학교,전북,전북대학교,본교,기존,국립,54896,"전라북도 전주시 덕진구 백제대로 567 (금암동, 전북대학교)",전라북도 전주시,덕진구,전라북도 전주시 덕진구
383,대학교,경기,가천대학교,본교,기존,사립,13120,"경기도 성남시 수정구 성남대로 1342 (복정동, 가천대학교)",경기도 성남시,수정구,경기도 성남시 수정구
437,대학교,세종,고려대학교 세종캠퍼스,분교1,기존,사립,30019,"세종특별자치시 조치원읍 세종로 2511 (서창리, 고려대학교세종캠퍼스)",세종특별자치시 세종시,세종로,세종특별자치시 세종시
603,대학교,세종,대전가톨릭대학교,본교,기존,사립,30006,"세종특별자치시 전의면 가톨릭대학로 30 (신방리, 대전가톨릭대학교)",세종특별자치시 세종시,가톨릭대학로,세종특별자치시 세종시
823,대학교,경기,성결대학교,본교,기존,사립,14097,"경기도 안양시 만안구 성결대학로 53 (안양동, 성결대학교)",경기도 안양시,만안구,경기도 안양시 만안구
896,대학교,경기,수원가톨릭대학교,본교,기존,사립,18332,"경기도 화성시 봉담읍 왕림1길 67 (왕림리, 수원가톨릭대학교)",경기도 화성시,봉담읍,경기도 화성시
898,대학교,경기,수원대학교,본교,기존,사립,18323,"경기도 화성시 봉담읍 와우안길 17 (와우리, 수원대학교)",경기도 화성시,봉담읍,경기도 화성시
961,대학교,경기,아세아연합신학대학교,본교,기존,사립,12508,"경기도 양평군 옥천면 경강로 1276 (아신리, 아세아연합신학대학교)",경기도 양평군,옥천면,경기도 양평군
984,대학교,경기,안양대학교,본교,기존,사립,14028,"경기도 안양시 만안구 삼덕로37번길 22 (안양동, 안양대학교)",경기도 안양시,만안구,경기도 안양시 만안구
1059,대학교,울산,울산대학교,본교,기존,사립,44610,"울산광역시 남구 대학로 93 (무거동, 울산대학교)",울산광역시 남구,대학로,울산광역시 남구


In [27]:
socar_region_univ.shape

(40, 11)

In [28]:
socar_region_univ.to_csv(path_dir + '/socar_region_univ_info.csv', index=False)

In [29]:
socar_region_univ = pd.read_csv(path_dir + '/socar_region_univ_info.csv')
socar_region_univ.shape

(40, 11)

In [30]:
g = socar_region_univ['위치'].value_counts().reset_index()
g = g.rename(columns={'index':'위치', '위치':'대학수'})